# Implementation of baseline using Artificial Neural Network with Original feature set and BSA

Necessary files: logistic_regression_functions.py, ann_functions.py, testing_functions.py, BRA.csv

In [ ]:
import logistic_regression_functions
import ann_functions
import testing_functions
import pandas as pd
from datetime import datetime
import requests
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tqdm import tqdm

The baseline is implemented according to the paper An Improved Prediction System for Football a Match Result bu C. P. Igiri and E. O. Nwachukwu from 2014. The original features as described in the paper (except for few ones which are not available for the BSA) are used and the model is evaluated on the BSA dataset.

The authors used only one season for training, so I did the same. I used seasons 2017 for training season 2018 as validation set and the second half of season 2019 for testing. The validation set is used to select the best checkpoint of the training according to the validation accuracy.

In [ ]:
full_dataset = pd.read_csv('BRA.csv')
full_dataset.rename(columns = {'Home':'HomeTeam', 'Away': 'AwayTeam',
                               'HG': 'FTHG', 'AG': 'FTAG', 'Res': 'FTR',
                               'PH': 'B365H', 'PD': 'B365D',
                               'PA': 'B365A'}, inplace = True)
for season in full_dataset['Season'].unique():
  dataset = full_dataset[full_dataset['Season'] == season]
  dataset.to_csv('BSA_' + str(season)[-2:] + '.csv', index=False)

In [ ]:
# Arguments for getting the attacks are not needed
results_train, matches_per_round = ann_functions.create_data_single('BSA_17.csv', None, None, None, None, include_stats=False)
results_val, matches_per_round = ann_functions.create_data_single('BSA_18.csv', None, None, None, None, include_stats=False)
# Dates are returned as well for dividing testing season into slices
results_test, matches_per_round = ann_functions.create_data_single('BSA_19.csv', None, None, None, None, include_stats=False,
                                  return_dates=True)
X_train = results_train.drop('FTR', axis=1)
y_train = results_train['FTR']
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']

In [ ]:
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

They didnt say what ANN architecture they used. I used 3 dense layers with 64, 32 and 16 neurons. I used dropout to overcome overfitting.

I always let the model train for 100 epochs and took the weights from the best epoch according to validation accuracy.

To use all the data available, the first half of the testing season was added to the training data.

In [ ]:
# Some rounds in the beginning are ignored, this is the correct index
# of the start of the second half of the season
start_test_index = 13 * matches_per_round

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

For evaluation on the testing dataset I divided the testing data into rounds approximately. The model is trained on the training dataset, then it is evaluated on one round of the testing dataset and this round is added to the training dataset.

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/14 [00:00<?, ?it/s]

1/1 [==============================] - 0s 138ms/step - loss: 1.1461 - accuracy: 0.2727


  7%|▋         | 1/14 [00:21<04:34, 21.13s/it]

1/1 [==============================] - 0s 140ms/step - loss: 0.8261 - accuracy: 0.6429


 14%|█▍        | 2/14 [00:42<04:12, 21.07s/it]

...

1/1 [==============================] - 0s 146ms/step - loss: 1.2762 - accuracy: 0.6923


 93%|█████████▎| 13/14 [06:24<00:30, 30.43s/it]

1/1 [==============================] - 0s 144ms/step - loss: 0.9789 - accuracy: 0.6471


100%|██████████| 14/14 [07:05<00:00, 30.43s/it]


0.5000000089406967


The testing accuracy is 50%.